# **KogSys-ML-B: Einführung in Maschinelles Lernen**
## **Deep Learning 2: Training Procedure**
---

To set up a new conda environment suitable for this notebook, you can use the following console commands:

```bash
conda create -y -n pytorch python=3.13
conda activate pytorch
python -m pip install -r requirements.txt
```

**Note**: Conda can become very hard-drive hungry when you use many environments. Consider regularly deleting environments you no longer need and running the ``conda clean --all`` command to remove no longer needed packages and cached files.

You can also install the requirements for this notebook into an existing environment by running the cell below:

In [23]:
# !python -m pip install -q -U -r requirements.txt

In [24]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

torch.manual_seed(2025)

### **1 Recap**

In last week's tutorial, we have seen the basic functionality of PyTorch and implemented our first own CNN! A possible solution for this is implemented here, so if you still want to catch up on last weeks exercises, don't look too closely at the cells from this section!

In [25]:
### choosing the device dynamically ###


def get_device() -> str:
    """
    Automatically checks if PyTorch has been installed for the use with CUDA (on NVIDIA GPUs) or MPS (Metal Performance Shaders, on Apple M chips). If neither is available, the CPU is used.
    """
    return (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )


get_device()

'mps'

In [26]:
### model ###


class Model(nn.Module):
    """
    The Convolutional Neural Network implemented last tutorial session.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.network = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=8,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(
                in_channels=8,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16 * 8 * 8, 10, bias=False),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        return self.network(x)

In [27]:
### data loading ###

train_data = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
test_data = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

### **2 Deconstructing the Basic Training Loop**

At the beggining of training, we initialize the Model to random weights (which is done by just calling the constructor) and moving it to the optimal device (which is set in one of the recap-cells).

In [28]:
### initialize model, move to device ###

model = Model()
model = model.to(get_device())

We then choose our loss function and optimizer. The loss function needs to be callable (in this case it is a callable object), and the optimizer is a `torch.optim.Optimizer` object, in this case Stochastic Gradient Descent. The optimizer must always be passed the parameters which it should optimize. We get those by calling `model.parameters()`.

In [29]:
### loss function and optimizer ###

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9)

In the training loop, we first iterate over epochs, and secondly over a dataloader – the training loader for training the model, obviously. We unpack the batch provided by the dataloader to labels and images (inputs). `optimizer.zero_grad()` sets the previously calculated gradients to zero. We then send the inputs to the models and calculate the loss function using the labels and model outputs. We then backpropagate the loss by calling `loss.backward()`, and optimize the parameters by calling `optimizer.step()`.

We record the batch `idx` using `enumerate` to be able to calculate average losses for recording training information.

At the end of training, we save the model to a file.

In [ ]:
### training loop ###

print("Starting Training")
model.train()

for epoch in range(10):  # Limit to 10 epochs to keep the runtime short
    sum_loss = 0.0

    for idx, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(get_device()), data[1].to(get_device())

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        if idx % 1000 == 999:
            print(f"Epoch {epoch + 1}, batch {idx + 1}: loss {sum_loss / 1000:.3f}")
            sum_loss = 0.0

print("Finished Training")


### save the model ###

model.eval()
torch.save(model.state_dict(), "cifar10_cnn.pth")

Starting Training
Epoch 1, batch 1000: loss 2.129
Epoch 1, batch 2000: loss 2.129
Epoch 1, batch 3000: loss 2.127


KeyboardInterrupt: 

The following code loads the weights from the save-file, so that you can repeat evaluation without re-training the model when coming back to this notebook.

In [10]:
### load the model ###

model = Model()
model.load_state_dict(torch.load("cifar10_cnn.pth", weights_only=True))
model = model.to(get_device())

In [11]:
### evaluate ###

def eval(data: torch.utils.data.DataLoader, model: nn.Module) -> float:
    """
    Calculates accuracy of `model` on the DataLoader `data`
    """
    model.eval()
    device = next(model.parameters()).device

    with torch.no_grad():
        true, pred = [], []
        for batch in data:
            images, labels = batch[0].to(device), batch[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            true.extend(labels.to("cpu"))
            pred.extend(predicted.to("cpu"))

    return accuracy_score(true, pred)


print(f"Test Accuracy: {eval(test_loader, model):.2f}")

Test Accuracy: 0.44


### **03 Building a Better Trianing Loop**

#### **03.1 Optimizers: `Adam`**

**What is Adam?** [Adam](https://paperswithcode.com/method/adam) is an optimization algorithm (Kingma & Ba, 2015). Since then, it has found wide application for optimizing neural network parameters. It extends Stochastic Gradient Descent with both Momentum (regulated by $\beta_1$) and Root Mean Square Propagation (RMSprop), which essentially adapts the learning rate for each to-be-optimized parameter individually (regulated by $\beta_2$).

In [12]:
del model
model = Model()  # Create new untrained model, so that we can use this optimizer in training later (otherwise we would have to copy-paste this code)

optimizer = torch.optim.Adam(
    params=model.parameters(),  # parameters to optimize
    lr=0.001,  # learning rate
    betas=(
        0.9,
        0.999,
    ),  # beta 1 momentum factor, beta 2 is RMSprop factor for per-parameter learning rate adjustment
    eps=1e-8,  # parameter avoiding RMSprop denominator collapse
    weight_decay=0,  # Factor to way in the L2 Norm (Euclidean distance) of all weights, i.e. not only minimize loss, but also weight values
)

#### **03.2 Learning Rate Scheduling**

Training performance can be increased by adapting the learning rate, with techniques like learning rate warmup (not starting with the full learning rate but increasing it over the first episodes) or decay (reducing the learning rate as training goes on). A very effective technique demonstrated for example in the [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) paper (He et al., 2016). See how the classification error decreases visibly after reducing the learning rate:

<image src='images/lr_schedule.png' style='width:800px'>

In [13]:
### learning rate scheduling ###

lr_scheduler = ReduceLROnPlateau(
    optimizer=optimizer,  # the optimizer for which learning rate should be adapted
    mode="max",  # whether the metric should increase or decrease. Default is 'min' to be used with loss, when used with acc should be 'max'
    patience=3,  # how long a metric must stop improving by at least best * (1 +/- threshold) per default, can also be set to absolute threshold mode
    threshold=1e-4,  # Threshold for patience calculation
    factor=0.1,  # The value to multiply the learning rate with
)

Learning rate scheduling should be applied after the optimizer's update by calling `lr_scheduler.step()`. This should usually happen after iterating through the training DataLoader. Note that the `ReduceLROnPlateau` scheduler's `step()` method must be passed a value to track, i.e. which must plateau for learning rate to be reduced.

#### **03.3 Validation**

To track training progress, looking only at the loss is not the best option. While we describe a performance criterion using the loss, it does not tell us as much about model _performance_ as say a calculated accuracy on a validation dataset would. Let's do exactly that!

We already have a function for calculating accuracy given a model and a DataLoader. Now we just need to split our dataset, and torch has a function exactly for that: `torch.utils.data.random_split()`. Note that this task (splitting an existing dataset) is different from the one presented in the assignment, where we want to build the split into the Dataset class itself.

**Note: Validation Dataset.** For datasets that are intended to be used as benchmarks, the test dataset is often either provided without labels or not provided at all. This is done to prevent models from being trained on the test data to cheat on the leaderboards. The validation dataset is thus all we have to get an estimate of how well our model performs.  
In such cases, we should not use the validation dataset for the classic validation tasks (e.g., to calculate metrics for early stopping), but rather treat it as our test dataset. Instead, it is advisable to create our own validation dataset from the training dataset.

In [14]:
### create validation set from train set ###

train, val = torch.utils.data.random_split(
    dataset=train_data,  # Dataset object to split
    lengths=(0.85, 0.15),  # Fractions of the returned datasets, must sum to 1
    generator=torch.Generator().manual_seed(
        2025
    ),  # Ensures reproducibility, optional parameter
)

train_loader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=8, shuffle=False)

#### **03.4 Checkpointing**

Using `torch.save()` you can save components of your model and training process by passing a `dictionary`. Such checkpoint files are by convention ending in either `.pth` or `.pt`.

In [15]:
### checkpointing ###

def save(save_dict: dict, path: str) -> None:
    """
    Wrapper around torch.save, to demonstrate the syntax. This function may be used in the loop, or its content itself.
    """

    torch.save(save_dict, path)

A common dictionary to save could be the following:

In [16]:
{
    "epoch": epoch,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "scheduler_state_dict": lr_scheduler.state_dict(),
    "loss": loss,
}

{'epoch': 9,
 'model_state_dict': OrderedDict([('network.0.weight',
               tensor([[[[ 9.9085e-02, -1.5615e-01, -5.1897e-02],
                         [-7.0170e-02, -1.5536e-01,  1.5801e-01],
                         [-7.2557e-03,  4.0359e-02, -2.3224e-02]],
               
                        [[ 1.2900e-01, -1.4678e-01,  1.1385e-01],
                         [ 1.7391e-02, -1.2739e-01, -8.1667e-02],
                         [ 8.1816e-02,  2.7578e-02, -7.0297e-03]],
               
                        [[ 5.2211e-02,  4.0076e-02,  1.7365e-02],
                         [-8.1381e-02, -1.0559e-01, -1.3982e-01],
                         [ 1.1293e-01,  1.9168e-01,  2.4398e-02]]],
               
               
                       [[[ 9.3824e-02,  1.4963e-01,  1.5513e-01],
                         [ 6.3655e-02,  8.4766e-02,  1.3340e-01],
                         [-8.3911e-02, -1.3094e-01, -1.3093e-01]],
               
                        [[-1.1492e-01, -1.3217e-01, -2.

...which can then be loaded using torch.load, and be accessed like the originally saved dictionary. Models, optimizers and schedulers have `.load_state_dict()` methods to load the stored `state_dict`s.

#### **03.5 Progress Bars!!!**

The `tqdm` Library offers progress bars for the console which display progress (duh) and can also show some information. We usually use progress bars around the DataLoaders rather than the epoch counter.

In [17]:
### default progress bar ###

for i in tqdm(range(1000000)):
    time.sleep(3e-6)

  0%|          | 0/1000000 [00:00<?, ?it/s]

100%|██████████| 1000000/1000000 [00:05<00:00, 173044.69it/s]


In [18]:
### customizable progress bar ###

pbar = tqdm(range(30000))

for i in pbar:
    pbar.set_description(f"i: {i}")

i: 29999: 100%|██████████| 30000/30000 [00:08<00:00, 3670.40it/s]


In [19]:
### syntactic sugar ###

for i in (pbar := tqdm(range(30000), ncols=100)):  # ncols sets the width of the bar
    pbar.set_description(f"i = {i}")

i = 29999: 100%|████████████████████████████████████████████| 30000/30000 [00:08<00:00, 3545.93it/s]


#### **03.6 Putting It All Together**

In [20]:
model.to(get_device())

for epoch in range(10):
    loss_sum = float(0)
    model.train()

    for batch in (pbar := tqdm(train_loader, ncols=100)):
        pbar.set_description(f"Epoch {epoch + 1}")

        inputs, labels = batch[0].to(get_device()), batch[1].to(get_device())

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    # validation:

    model.eval()
    mean_loss = loss_sum / len(train_loader)
    val_acc = eval(val_loader, model)

    # lr scheduling:

    lr_scheduler.step(val_acc)

    print(
        f"Epoch {epoch + 1}:\tlr = {lr_scheduler.get_last_lr()[0]},\tmean_loss = {mean_loss:.4f},\tval_acc = {val_acc:.4f}"
    )

    # checkpoint every 5 epochs:

    if (epoch + 1) % 5 == 0:
        save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "loss": mean_loss,
            },
            f"checkpoint_epoch_{epoch + 1}.pth",
        )

Epoch 1: 100%|█████████████████████████████████████████████████| 5313/5313 [00:17<00:00, 296.12it/s]


Epoch 1:	lr = 0.001,	mean_loss = 2.0698,	val_acc = 0.4349


Epoch 2: 100%|█████████████████████████████████████████████████| 5313/5313 [00:17<00:00, 311.46it/s]


Epoch 2:	lr = 0.001,	mean_loss = 1.9953,	val_acc = 0.4833


Epoch 3: 100%|█████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 381.89it/s]


...

Epoch 8:	lr = 0.001,	mean_loss = 1.8677,	val_acc = 0.5533


Epoch 9: 100%|█████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 380.18it/s]


Epoch 9:	lr = 0.001,	mean_loss = 1.8586,	val_acc = 0.5472


Epoch 10: 100%|████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 383.26it/s]


Epoch 10:	lr = 0.001,	mean_loss = 1.8518,	val_acc = 0.5712


In [21]:
checkpoint = torch.load("checkpoint_epoch_10.pth", weights_only=True)

epoch = checkpoint["epoch"]

model = Model()
model.load_state_dict(checkpoint["model_state_dict"])
model.to(get_device())

optimizer = torch.optim.Adam(model.parameters())
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
lr_scheduler.load_state_dict(checkpoint["lr_scheduler"])


for epoch in range(epoch + 1, 20):
    loss_sum = float(0)
    model.train()

    for batch in (pbar := tqdm(train_loader, ncols=100)):
        pbar.set_description(f"Epoch {epoch + 1}")

        inputs, labels = batch[0].to(get_device()), batch[1].to(get_device())

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    # validation:

    model.eval()
    mean_loss = loss_sum / len(train_loader)
    val_acc = eval(val_loader, model)

    # lr scheduling:

    lr_scheduler.step(val_acc)

    print(
        f"Epoch {epoch + 1}:\tlr = {lr_scheduler.get_last_lr()[0]},\tmean_loss = {mean_loss:.4f},\tval_acc = {val_acc:.4f}"
    )

    # checkpoint every 5 epochs:

    if (epoch + 1) % 5 == 0:
        save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "loss": loss,
            },
            f"checkpoint_epoch_{epoch + 1}.pth",
        )

Epoch 11: 100%|████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 385.91it/s]


Epoch 11:	lr = 0.001,	mean_loss = 1.8457,	val_acc = 0.5825


Epoch 12: 100%|████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 382.52it/s]


Epoch 12:	lr = 0.001,	mean_loss = 1.8396,	val_acc = 0.5813


Epoch 13: 100%|████████████████████████████████████████████████| 5313/5313 [00:14<00:00, 376.98it/s]


...

Epoch 18:	lr = 0.001,	mean_loss = 1.8196,	val_acc = 0.5887


Epoch 19: 100%|████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 380.82it/s]


Epoch 19:	lr = 0.001,	mean_loss = 1.8169,	val_acc = 0.5856


Epoch 20: 100%|████████████████████████████████████████████████| 5313/5313 [00:13<00:00, 381.25it/s]


Epoch 20:	lr = 0.001,	mean_loss = 1.8157,	val_acc = 0.5720


In [22]:
eval(train_loader, model)

0.618964705882353

Note in the case above the model produced after epoch 20 may not be the best model we have seen during training, judging by the validation accuracy. For all we know, a model from a previous epoch should perform better. For this purpose, we may also introduce a second set of checkpoints, which always store (and overwrite) a model whenever a new maximum validation accuracy is reached.

### **What Else?**

Of course, this isn't everything to learn about PyTorch! Here is an (incomplete) list of resources for you to look at if you want to dive deeper into this framework! Note that some of these are really advanced – so don't worry if you don't understand them, you won't need them for this course.

- [Tensorboard visualization of training metrics](https://docs.pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html)
- [Distributed Data Parallel (DDP), i.e. Multi-GPU Training](https://docs.pytorch.org/tutorials/intermediate/ddp_tutorial.html)
- Technically not PyTorch, but an important tool for academic experiments: Running experiments from configs, e.g. via [YACS](https://github.com/rbgirshick/yacs) or [JSON argparse](https://jsonargparse.readthedocs.io/en/v4.44.0/)

### **Bibliography**

Kingma, D. P., & Ba, J. (2015). Adam: A Method for Stochastic Optimization. In Y. Bengio & Y. LeCun (Hrsg.), 3rd international conference on learning representations, ICLR 2015, san diego, CA, USA, may 7-9, 2015, conference track proceedings. http://arxiv.org/abs/1412.6980

He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep Residual Learning for Image Recognition. 770–778. https://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html